In [136]:
import pandas as pd
from datetime import datetime, timedelta
pd.set_option('display.max_columns', None)

In [137]:
df_stops = pd.read_csv('data/stops.txt')
df_routes = pd.read_csv('data/routes.txt')
df_stoptimes = pd.read_csv('data/stop_times.txt')
df_trips = pd.read_csv('data/trips.txt')
df_shapes = pd.read_csv('data/shapes.txt')
df_dates = pd.read_csv('data/calendar_dates.txt')

In [138]:
columns_to_drop = ['stop_headsign', 'stop_code', 'route_long_name', 'route_desc', 'route_color', 'route_text_color', 'trip_short_name']

In [139]:
result = pd.merge(df_stoptimes, df_stops, how='inner',on='stop_id')
result = pd.merge(result, df_trips, how='inner',on='trip_id')
result = pd.merge(result, df_routes, how='inner',on='route_id')
result = pd.merge(result, df_dates, how='inner',on='service_id')

In [140]:
result = result.drop(columns=columns_to_drop)

In [141]:
result['arrival_time_minutes'] = result['arrival_time'].map(lambda x: int(x[3:5]))
result['arrival_time_hours'] = result['arrival_time'].map(lambda x: int(x[:2]))
result['departure_time_minutes'] = result['departure_time'].map(lambda x: int(x[3:5]))
result['departure_time_hours'] = result['departure_time'].map(lambda x: int(x[:2]))

result['arrival_time_hours'] = result['arrival_time_hours'].apply(lambda x: x-24 if x >= 24 else x )
result['departure_time_hours'] = result['departure_time_hours'].apply(lambda x: x-24 if x >= 24 else x )

result.date = result.date.astype('str')
result['month'] = result['date'].map(lambda x: x[4:6])
result['day'] = result['date'].map(lambda x: x[6:8])
result['year'] = result['date'].map(lambda x: x[0:4])
result['date2'] = result['year'] + '-' +  result['month'] + '-' + result['day']

result['departure_time'] = result['date2'] + ' ' + result['departure_time']
result['arrival_time'] = result['date2'] + ' ' + result['arrival_time']

result['departure_time'] = result['departure_time'].apply(lambda x: x[:8] + (str(int(x[8:10]) + 1) if int(x[11:13]) >= 24 else x) + x[10:11] + str(int(x[11:13]) - 24) + x[13:] if int(x[11:13]) >= 24 else x)
result['arrival_time'] = result['arrival_time'].apply(lambda x: x[:8] + (str(int(x[8:10]) + 1) if int(x[11:13]) >= 24 else x) + x[10:11] + str(int(x[11:13]) - 24) + x[13:] if int(x[11:13]) >= 24 else x)

result['arrival_time'] = result['arrival_time'].apply(lambda x: datetime.strptime(x,'%Y-%m-%d %H:%M:%S'))
result['departure_time'] = result['departure_time'].apply(lambda x: datetime.strptime(x,'%Y-%m-%d %H:%M:%S'))

<h1>Functions</h1>

In [148]:
def get_all_departures_from_bus_stop(stop_id: int, arrival_time: str):
    time = datetime.strptime(arrival_time, '%Y-%m-%d %H:%M')
    all_deparutres = result[(result["stop_id"]==stop_id) &
       (result['departure_time'] >= time) & (result['departure_time'] <= time + timedelta(hours=24)) &
       (result['drop_off_type'] == 0) & (result['pickup_type'] == 1)]
    return all_deparutres
#df.sort_values(by=['col1', 'col2'])

In [149]:
tmp = get_all_departures_from_bus_stop(313,'2023-01-15 00:45')
tmp.sort_values(by=['departure_time'])

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,pickup_type,drop_off_type,stop_name,stop_lat,stop_lon,route_id,service_id,trip_headsign,direction_id,shape_id,wheelchair_accessible,agency_id,route_short_name,route_type,date,exception_type,arrival_time_minutes,arrival_time_hours,departure_time_minutes,departure_time_hours,month,day,year,date2
219914,403202301142358_101_403-18,2023-01-15 01:02:00,2023-01-15 01:02:00,313,55,1,0,Wrzeszcz PKP 06,54.381766,18.603963,403,120230114,Wrzeszcz PKP,0,403_903388,1,0,N3,700,20230114,1,2,1,2,1,01,14,2023,2023-01-14
219534,403202301140058_101_403-15,2023-01-15 02:02:00,2023-01-15 02:02:00,313,55,1,0,Wrzeszcz PKP 06,54.381766,18.603963,403,120230114,Wrzeszcz PKP,0,403_903388,1,0,N3,700,20230114,1,2,2,2,2,01,14,2023,2023-01-14
219706,403202301150158_101_403-16,2023-01-15 03:02:00,2023-01-15 03:02:00,313,55,1,0,Wrzeszcz PKP 06,54.381766,18.603963,403,120230114,Wrzeszcz PKP,0,403_903388,1,0,N3,700,20230114,1,2,3,2,3,01,14,2023,2023-01-14
219868,403202301150258_111_403-17,2023-01-15 03:53:00,2023-01-15 03:53:00,313,45,1,0,Wrzeszcz PKP 06,54.381766,18.603963,403,120230114,Wrzeszcz PKP,0,403_903537,1,0,N3,700,20230114,1,53,3,53,3,01,14,2023,2023-01-14
220018,403202301150358_111_403-18,2023-01-15 04:53:00,2023-01-15 04:53:00,313,45,1,0,Wrzeszcz PKP 06,54.381766,18.603963,403,120230114,Wrzeszcz PKP,0,403_903537,1,0,N3,700,20230114,1,53,4,53,4,01,14,2023,2023-01-14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
280932,116202301152242_12_116-01,2023-01-15 22:59:00,2023-01-15 22:59:00,313,12,1,0,Wrzeszcz PKP 06,54.381766,18.603963,116,120230115,Wrzeszcz PKP,1,116_903424,1,0,116,700,20230115,1,59,22,59,22,01,15,2023,2023-01-15
279509,110202301152246_22_110-41,2023-01-15 23:05:00,2023-01-15 23:05:00,313,13,1,0,Wrzeszcz PKP 06,54.381766,18.603963,110,120230115,Wrzeszcz PKP,1,110_903587,1,0,110,700,20230115,1,5,23,5,23,01,15,2023,2023-01-15
243325,162202301152247_22_162-54,2023-01-15 23:16:00,2023-01-15 23:16:00,313,19,1,0,Wrzeszcz PKP 06,54.381766,18.603963,162,120230115,Wrzeszcz PKP,1,162_903467,1,0,162,700,20230115,1,16,23,16,23,01,15,2023,2023-01-15
280200,157202301152312_32_157-51,2023-01-15 23:33:00,2023-01-15 23:33:00,313,17,1,0,Wrzeszcz PKP 06,54.381766,18.603963,157,120230115,Wrzeszcz PKP,1,157_903688,1,0,157,700,20230115,1,33,23,33,23,01,15,2023,2023-01-15


<h1>Tests</h1>

In [150]:
result[(result['drop_off_type'] == 0) & (result['pickup_type'] == 1)]

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,pickup_type,drop_off_type,stop_name,stop_lat,stop_lon,route_id,service_id,trip_headsign,direction_id,shape_id,wheelchair_accessible,agency_id,route_short_name,route_type,date,exception_type,arrival_time_minutes,arrival_time_hours,departure_time_minutes,departure_time_hours,month,day,year,date2
43,401202301122334_152_401-01,2023-01-13 00:43:00,2023-01-13 00:43:00,14555,43,1,0,Osowa PKP 01,54.420956,18.452618,401,120230112,Osowa PKP,1,401_870029,1,0,N1,700,20230112,1,43,0,43,0,01,12,2023,2023-01-12
45,401202301122258_301_401-01,2023-01-12 23:25:00,2023-01-12 23:25:00,195,22,1,0,Jaworzniaków 01,54.318041,18.568734,401,120230112,Jaworzniaków,0,401_870024,1,0,N1,700,20230112,1,25,23,25,23,01,12,2023,2023-01-12
68,401202301130113_11_401-01,2023-01-13 02:25:00,2023-01-13 02:25:00,195,49,1,0,Jaworzniaków 01,54.318041,18.568734,401,120230112,Jaworzniaków,0,401_870017,1,0,N1,700,20230112,1,25,2,25,2,01,12,2023,2023-01-12
161,401202301130230_392_401-01,2023-01-13 03:43:00,2023-01-13 03:43:00,14555,47,1,0,Osowa PKP 01,54.420956,18.452618,401,120230112,Osowa PKP,1,401_870018,1,0,N1,700,20230112,1,43,3,43,3,01,12,2023,2023-01-12
190,401202301130357_322_401-04,2023-01-13 04:41:00,2023-01-13 04:41:00,1910,29,1,0,Grudziądzka 04,54.396338,18.622572,401,120230112,Grudziądzka,1,401_870033,1,0,N1,700,20230112,1,41,4,41,4,01,12,2023,2023-01-12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2051133,4202301261501_42_004-12,2023-01-26 15:57:00,2023-01-26 15:57:00,201,36,1,0,Jelitkowo 02,54.423228,18.592695,4,220230126,Jelitkowo,1,4_884464,1,0,4,900,20230126,1,57,15,57,15,01,26,2023,2023-01-26
2051170,4202301260901_42_004-14,2023-01-26 09:57:00,2023-01-26 09:57:00,201,36,1,0,Jelitkowo 02,54.423228,18.592695,4,220230126,Jelitkowo,1,4_884464,1,0,4,900,20230126,1,57,9,57,9,01,26,2023,2023-01-26
2051207,4202301261121_42_004-14,2023-01-26 12:17:00,2023-01-26 12:17:00,201,36,1,0,Jelitkowo 02,54.423228,18.592695,4,220230126,Jelitkowo,1,4_884464,1,0,4,900,20230126,1,17,12,17,12,01,26,2023,2023-01-26
2051244,4202301261341_42_004-14,2023-01-26 14:37:00,2023-01-26 14:37:00,201,36,1,0,Jelitkowo 02,54.423228,18.592695,4,220230126,Jelitkowo,1,4_884464,1,0,4,900,20230126,1,37,14,37,14,01,26,2023,2023-01-26
